Pydantic adds run-time validation

In [1]:
from typing import Literal, Annotated
from pydantic import BaseModel, Field, ConfigDict


AttributeName = Literal[
    "Might",
    "Agility",
    "Vitality",
    "Insight",
    "Arcana",
    "Presence",
]

AttributeValue = Annotated[int, Field(description="Attribute value can be between 1 and 100",ge=1, le=100)]

class Attributes(BaseModel):
    model_config = ConfigDict(validate_assignment=True)
    
    Might: AttributeValue
    Agility: AttributeValue
    Vitality: AttributeValue
    Insight: AttributeValue
    Arcana: AttributeValue
    Presence: AttributeValue


class Character(BaseModel):
    name: str
    attributes: Attributes


def update_attribute(
    character: Character,
    attribute_name: AttributeName,
    delta: int,
) -> Character:
    current_value = getattr(character.attributes, attribute_name)
    new_value = current_value + delta

    # Re-assignment triggers Pydantic validation
    setattr(character.attributes, attribute_name, new_value)

    return character

character = Character(
    name="Aelar",
    attributes=Attributes(
        Might=10,
        Agility=8,
        Vitality=12,
        Insight=14,
        Arcana=16,
        Presence=11,
    ),
)

Real world example

In [ ]:

character = Character.model_validate({
    "name": "Aelar",
    "attributes": {
        "might": 90,
        "Agility": 8,
        "Vitality": 12,
        "Insight": 14,
        "Arcana": 16,
        "Presence": 11,
        "tset": 123,
    }
}
)

print(character)

Now with Pydantic our Domain Data standards are protected at run-time
- Invalid data is blocked at the boundaries of our system

In [2]:
update_attribute(character, "Might", 100)

ValidationError: 1 validation error for Attributes
Might
  Input should be less than or equal to 100 [type=less_than_equal, input_value=110, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/less_than_equal